# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:
- population - The population as of 2015.
- population_growth - The annual population growth rate, as a percentage.
- area - The total land and water area.

We'll use SQL in Jupyter notebook to explore and analyze data.

### Introduction

Use the following code to connect our Jupyter notebook to dataquest database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

### Overview of the Data

write a query to return information on the tables in the database

In [2]:
%%sql
SELECT *
   FROM sqlite_master
    WHERE type='table'

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Write another query returning the first 5 rows of the `facts` table in the database.

In [3]:
%%sql
SELECT * 
   FROM facts 
    LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in [square kilometers](https://www.cia.gov/library/publications/the-world-factbook/rankorder/2147rank.html).
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

Write a single query to return the following:
- Min. population
- Max. population
- Min. population growth
- Max. population growth

In [4]:
%%sql
SELECT MIN(population), 
       MAX(population), 
       MIN(population_growth), 
       MAX(population_growth)
    FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


As you'll see, minimum of population is 0 and maximum of population is `7256490011`(7.2 billion people which is almost total population on earth) which we could suspect them as incorrect data. 

Minimum of population growth could be 0 or negative value if birth rate is less than death rate, so we'll keep observing the data to determine if this is correct.


### Exploring Outliers
Write a query to return the country with the minimum population

In [5]:
%%sql
SELECT name 
   FROM facts 
    WHERE population == (SELECT MIN(population) 
                         FROM facts
                        );

 * sqlite:///factbook.db
Done.


name
Antarctica


Write a query to return country with the maximum population

In [6]:
%%sql
SELECT name 
   FROM facts 
    WHERE population == (SELECT MAX(population) 
                         FROM facts
                        );

 * sqlite:///factbook.db
Done.


name
World


The country of the minimum population is Antartica which can explain 0 population. If you see CIA Factbook [page of Antartica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html), there are no indigenous inhabitants but only permenent and summer-only reaserch staffs.

![Antartica](https://s3.amazonaws.com/dq-content/257/fb_antarctica.png)

The country of the maximum population is World which means the total of countries and can explain population of over 7.2 billion.



### Summary Statistics

In [7]:
%%sql
SELECT MIN(population) AS 'min_pop',
       MAX(population) AS 'max_pop',
       MIN(population_growth) AS 'min_pop_growth',
       MAX(population_growth) AS 'max_pop_growth',
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
(sqlite3.OperationalError) near "FROM": syntax error
[SQL: SELECT MIN(population) AS 'min_pop',
       MAX(population) AS 'max_pop',
       MIN(population_growth) AS 'min_pop_growth',
       MAX(population_growth) AS 'max_pop_growth',
    FROM facts
    WHERE name != 'World';]
(Background on this error at: http://sqlalche.me/e/e3q8)


Given the exclusion of which country name is World, the maximum population is now 1.3 billion.

### Exploring Average Population and Area

write a query to return the average value of population and area

In [8]:
%%sql
SELECT ROUND(AVG(population),2) AS 'avg_population', 
       ROUND(AVG(area),2) AS 'avg_area'
    FROM facts;

 * sqlite:///factbook.db
Done.


avg_population,avg_area
62094928.32,555093.55


To finish, we'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have:

- Above average values for population.
- Below average values for area.

### Finding Densely Populated Countries

write a query to return all countries meeting both of the following conditions:
- population above average
- area below average

In [9]:
%%sql
SELECT * 
    FROM facts 
    WHERE population > (SELECT AVG(population) 
                        FROM facts) 
    AND area < (SELECT AVG(area)
               FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


### Next Steps

##### What country has the most people? What country has the highest growth rate?

In [10]:
%%sql
SELECT name 'Country',
       MAX(population) 'MAX pop'
    FROM facts
    WHERE Country != 'World';

 * sqlite:///factbook.db
Done.


Country,MAX pop
China,1367485388


In [11]:
%%sql
SELECT name 'Country',
       MAX(population_growth) 'MAX pop growth'
    FROM facts;

 * sqlite:///factbook.db
Done.


Country,MAX pop growth
South Sudan,4.02


China is the country with maxiized populations so far, South Sudan is the country with maximized population growth rate.

##### Which countries have the highest ratios of water to land?

In [12]:
%%sql
SELECT name 'Country', 
       MAX(area_water/area_land) 'Ratio of water to land',
       area_water,
       area_land
    FROM facts;

 * sqlite:///factbook.db
Done.


Country,Ratio of water to land,area_water,area_land
British Indian Ocean Territory,905,54340,60


British Indian Ocean Territory
![BIOT](https://www.cia.gov/library/publications/the-world-factbook/attachments/locator-maps/IO-locator-map.gif)


##### Which countries have more water than land?

In [13]:
%%sql
SELECT name 'Country', area_water, area_land FROM facts WHERE area_water>area_land

 * sqlite:///factbook.db
Done.


Country,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


Virgin Islands
![VI](https://www.cia.gov/library/publications/the-world-factbook/attachments/locator-maps/VQ-locator-map.gif)

##### Which countries will add the most people to their population next year?

In [14]:
%%sql
SELECT name 'Country', 
      population*population_growth/100 'Population_added_next_year' 
    FROM facts 
    WHERE name != 'World'
    ORDER BY Population_added_next_year DESC
    LIMIT 10;


 * sqlite:///factbook.db
Done.


Country,Population_added_next_year
India,15270686.1248
China,6153684.246
Nigeria,4448270.372
Pakistan,2906653.3662
Ethiopia,2874562.1691
Bangladesh,2703323.92
United States,2506677.1392
Indonesia,2355141.8008000003
"Congo, Democratic Republic of the",1944690.8320000002
Philippines,1626073.8536


##### Which countries have a higher death rate than birth rate?


In [15]:
%%sql
SELECT name 'Country', 
       birth_rate, 
       death_rate
    FROM facts
    WHERE death_rate > birth_rate
    LIMIT 10;

 * sqlite:///factbook.db
Done.


Country,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


##### What countries have the highest `population/area` ratio and how does it compare to list we found in the previous question?

In [16]:
%%sql
SELECT name 'Country',
       population/area,
       population,
       area
    FROM facts
    ORDER BY population/area DESC
    LIMIT 10;
       

 * sqlite:///factbook.db
Done.


Country,population/area,population,area
Macau,21168,592731,28
Monaco,15267,30535,2
Singapore,8141,5674472,697
Hong Kong,6445,7141106,1108
Gaza Strip,5191,1869055,360
Gibraltar,4876,29258,6
Bahrain,1771,1346613,760
Maldives,1319,393253,298
Malta,1310,413965,316
Bermuda,1299,70196,54


In [17]:
%%sql
SELECT ROUND(AVG(population),2)
    FROM facts;

 * sqlite:///factbook.db
Done.


"ROUND(AVG(population),2)"
62094928.32


The countries of highest population/area ratio are Macau, Monaco and Singapore. 

In the previous question, we filter the country whose population is above average and area is below average, this will make us neglect some countries below 62 millions population like countries listed above.